In [4]:
#creating and simulating a simple qubit

import cirq

def main():
    # Pick a qubit
    qubit = cirq.GridQubit(0,0)
    
    # Create a circuit
    circuit = cirq.Circuit(
    cirq.X(qubit)**0.5,    # square root of NOT
    cirq.measure(qubit, key = "m")    # measurement
    )
    print("Circuit:")
    print(circuit)
    
    # Simulate the circuit several times
    simulator = cirq.Simulator()
    result = simulator.run(circuit, repetitions = 20)
    print("Results:")
    print(result)
    
if __name__ == '__main__':
    main()
    

Circuit:
(0, 0): ───X^0.5───M('m')───
Results:
m=00000101010000110110


In [11]:
# Deutsch algorithm

import random
import cirq
from cirq import H, X, CNOT, measure

def main():
    # Choose qubits to use
    q0, q1 = cirq.LineQubit.range(2)
    
    # Pick a secret 2-bit function and create a circuit to query the oracle
    secret_function = [random.randint(0,12) for _ in range(2)]
    oracle = make_oracle(q0, q1, secret_function)
    print('Secret function:\nf(x) = <{}>'.format(
    ','.join(str(e) for e in secret_function)))
    
    # Embed the oracle into a quantum circuit querying it exactly once
    circuit = make_deutsch_circuit(q0, q1, oracle)
    print("Circuit:")
    print(circuit)
        
    # Simulate the circuit
    simulator = cirq.Simulator()
    result = simulator.run(circuit)
    print('Result of f(0)⊕f(1):')
    print(result)
    
def make_oracle(q0, q1, secret_function):
    """ Gates implementing the secret function f(x)"""
    
    # coverage: ignore
    if secret_function[0]:
        yield[CNOT(q0, q1), X(q1)]
    
    if secret_function[1]:
        yield CNOT(q0, q1)
    
    
def make_deutsch_circuit(q0, q1, oracle):
    c = cirq.Circuit()
    
    # Initialize qubits
    c.append([X(q1), H(q0)])
    
    # Query oracle
    c.append(oracle)
    
    # Measure in X basis
    c.append([H(q0), measure(q0, key = 'result')])
    return c

if __name__ == '__main__':
    main()
    

Secret function:
f(x) = <0,4>
Circuit:
0: ───H───@───H───M('result')───
          │
1: ───X───X─────────────────────
Result of f(0)⊕f(1):
result=1


In [24]:
"""The Bernstein-Vazirani algorithm.

The (non-recursive) BV algo takes a black-box oracle 
                    implementing a function:
                    
                    f(a) = a * factors + bias (mod 2)
        
                where 'bias' is 0 or 1 and 'factors' are vectors 
                with all elements = 0 or 1
                and the algo solves for 'factors' in a single query to oracle
        
"""

# Bernstein-Vazirani algorithm

import random
import cirq

def main(qubit_count = 8):
    circuit_sample_count = 3
    
    # choose qubits to use
    input_qubits = [cirq.GridQubit(i, 0) for i in range(qubit_count)]
    output_qubit = cirq.GridQubit(qubit_count, 0)
    
    # pick coefficients for oracle and create circuit to query it
    secret_bias_bit = random.randint(0, 1)
    secret_factor_bits = [random.randint(0, 1) for _ in range(qubit_count)]
    oracle = make_oracle(input_qubits,
                        output_qubit,
                        secret_factor_bits,
                        secret_bias_bit)
    print('Secret function:\nf(a) = a * <{}> + {} (mod 2)'.format(
    ','.join(str(e) for e in secret_factor_bits),
    secret_bias_bit))
    
    # embed oracle into a quantum circuit & query once
    circuit = bv_circuit(
        input_qubits, output_qubit, oracle)
    print('Circuit:')
    print(circuit)
    
    # sample from circuit a few times
    simulator = cirq.Simulator()
    result = simulator.run(circuit, repetitions = circuit_sample_count)
    frequencies = result.histogram(key = 'result', fold_func = bitstring)
    print("Sampled results:\n{}".format(frequencies))
    
    # Check if we actually found the secret value
    most_common_bitstring = frequencies.most_common(1)[0][0]
    print("Most common matches secret factors:\n{}".format(
        most_common_bitstring == bitstring(secret_factor_bits)))
    
def make_oracle(input_qubits,
               output_qubit,
               secret_factor_bits,
               secret_bias_bit):
    """Gate implementing the function f(a) = a*facotrs + bias (mod 2) """
    
    if secret_bias_bit:
        yield cirq.X(output_qubit)
        
    for qubit, bit in zip(input_qubits, secret_factor_bits):
        if bit:
            yield cirq.CNOT(qubit, output_qubit)
            

def bv_circuit(input_qubits, output_qubit, oracle):
    """solves for factors in f(a) = ..."""
    
    c = cirq.Circuit()
    
    # initialize qubits
    c.append([
        cirq.X(output_qubit),
        cirq.H(output_qubit),
        cirq.H.on_each(*input_qubits)
    ])
    
    # query oracle
    c.append(oracle)
    
    # measure in X basis
    c.append([
        cirq.H.on_each(*input_qubits),
        cirq.measure(*input_qubits, key = "result")
    ])
    
    return c

def bitstring(bits):
    return ''.join(str(int(b)) for b in bits)

if __name__ == '__main__':
    main()

Secret function:
f(a) = a * <1,1,0,0,0,1,1,0> + 1 (mod 2)
Circuit:
(0, 0): ───H───────────@───H───────────────M('result')───
                       │                   │
(1, 0): ───H───────────┼───@───H───────────M─────────────
                       │   │               │
(2, 0): ───H───H───────┼───┼───────────────M─────────────
                       │   │               │
(3, 0): ───H───H───────┼───┼───────────────M─────────────
                       │   │               │
(4, 0): ───H───H───────┼───┼───────────────M─────────────
                       │   │               │
(5, 0): ───H───────────┼───┼───@───H───────M─────────────
                       │   │   │           │
(6, 0): ───H───────────┼───┼───┼───@───H───M─────────────
                       │   │   │   │       │
(7, 0): ───H───H───────┼───┼───┼───┼───────M─────────────
                       │   │   │   │
(8, 0): ───X───H───X───X───X───X───X─────────────────────
Sampled results:
Counter({'11000110': 3})
Most common match

In [34]:
# Bell Inequality Test
""" this algo creates and simulates a circuit
    equivalent to a Bell Inequality test"""

import numpy as np
import cirq

def main():
    #create circuit
    circuit = bell_circuit()
    print('Circuit', circuit)
    
    #run simulations
#    print()
#    repetitions = 75
#    print('Simulating {} repetitions...'.format(repetitions))
#    result = cirq.Simulator().run(program = circuit, repetitions = repetitions)
    print()
    repetitions = 75
    print('Simulating {} repetitions...'.format(repetitions))
    result = cirq.Simulator().run(program=circuit, repetitions=repetitions)

    
    #get results
    a = np.array(result.measurements['a'][:, 0])
    b = np.array(result.measurements['b'][:, 0])
    x = np.array(result.measurements['x'][:, 0])
    y = np.array(result.measurements['y'][:, 0])
    outcomes = a^b == x & y
    win_percent = len([e for e in outcomes if e]) * 100 / repetitions
    
    #print data
    print()
    print('Results')
    print('a:', bitstrings(a))
    print('b:', bitstrings(b))
    print('x:', bitstrings(x))
    print('y:', bitstrings(y))
    print('(a XOR b) == (x AND y):\n ', bitstring(outcomes))
    print('Win rate:{}%'.format(win_percent))


def bell_circuit():
    #some definitions
    alice = cirq.GridQubit(0, 0)
    bob = cirq.GridQubit(1, 0)
    alice_referee = cirq.GridQubit(0, 1)
    bob_referee = cirq.GridQubit(1, 1)
    
    circuit = cirq.Circuit()
    
    #prepare shared entangled state
    circuit.append([
        cirq.H(alice),
        cirq.CNOT(alice, bob), 
        cirq.X(alice)**0.25,
    ])
    
    #referees flip coins
    circuit.append([
        cirq.H(alice_referee),
        cirq.H(bob_referee),
    ])
    
    #players do sqrt(X) based on their ref coin
    circuit.append([
        cirq.CNOT(alice_referee, alice)**0.5,
        cirq.CNOT(bob_referee, bob)**0.5        
    ])
    
    #then results are recorded
    circuit.append([
        cirq.measure(alice, key = 'a'),
        cirq.measure(bob, key = 'b'),
        cirq.measure(alice_referee, key = 'x'),
        cirq.measure(bob_referee, key = 'y')
    ])
    
    
def bitstring(bits):
    return ''.join('1' if e else '_' for e in bits)

if __name__ == '__main__':
    main()

Circuit None

Simulating 75 repetitions...


AttributeError: 'NoneType' object has no attribute 'has_measurements'

In [35]:
import numpy as np

import cirq


def main():
    # Create circuit.
    circuit = make_bell_test_circuit()
    print('Circuit:')
    print(circuit)

    # Run simulations.
    print()
    repetitions = 75
    print('Simulating {} repetitions...'.format(repetitions))
    result = cirq.Simulator().run(program=circuit, repetitions=repetitions)

    # Collect results.
    a = np.array(result.measurements['a'][:, 0])
    b = np.array(result.measurements['b'][:, 0])
    x = np.array(result.measurements['x'][:, 0])
    y = np.array(result.measurements['y'][:, 0])
    outcomes = a ^ b == x & y
    win_percent = len([e for e in outcomes if e]) * 100 / repetitions

    # Print data.
    print()
    print('Results')
    print('a:', bitstring(a))
    print('b:', bitstring(b))
    print('x:', bitstring(x))
    print('y:', bitstring(y))
    print('(a XOR b) == (x AND y):\n  ', bitstring(outcomes))
    print('Win rate: {}%'.format(win_percent))


def make_bell_test_circuit():
    alice = cirq.GridQubit(0, 0)
    bob = cirq.GridQubit(1, 0)
    alice_referee = cirq.GridQubit(0, 1)
    bob_referee = cirq.GridQubit(1, 1)

    circuit = cirq.Circuit()

    # Prepare shared entangled state.
    circuit.append([
        cirq.H(alice),
        cirq.CNOT(alice, bob),
        cirq.X(alice)**-0.25,
    ])

    # Referees flip coins.
    circuit.append([
        cirq.H(alice_referee),
        cirq.H(bob_referee),
    ])

    # Players do a sqrt(X) based on their referee's coin.
    circuit.append([
        cirq.CNOT(alice_referee, alice)**0.5,
        cirq.CNOT(bob_referee, bob)**0.5,
    ])

    # Then results are recorded.
    circuit.append([
        cirq.measure(alice, key='a'),
        cirq.measure(bob, key='b'),
        cirq.measure(alice_referee, key='x'),
        cirq.measure(bob_referee, key='y'),
    ])

    return circuit


def bitstring(bits):
    return ''.join('1' if e else '_' for e in bits)


if __name__ == '__main__':
    main()

Circuit:
(0, 0): ───H───@───X^-0.25───X────────M('a')───
               │             │
(0, 1): ───H───┼─────────────@^0.5────M('x')───
               │
(1, 0): ───────X───X─────────M('b')────────────
                   │
(1, 1): ───H───────@^0.5─────M('y')────────────

Simulating 75 repetitions...

Results
a: _111_111__1__11_1___11___1___1__1_1_111111_1___11___1_11__1_111__111_11__1_
b: _1111_11__1__1111___11___1__11_1_11__1_11_111_111___11_1____111__1___11_111
x: _1111_11______11_1____11____1_1111__1_11__1___1_1____11_1_1____11__1__111_1
y: ___11____1_11__11_1111_1____1__1111_1_1___1_1_1__111_11___11___1_1111___111
(a XOR b) == (x AND y):
   111_1_11111111111111111_11111111111111111_11_111111111111111111_11_11111111
Win rate: 90.66666666666667%
